In [6]:
import math as np
from scipy.stats import norm
# S: Spot Price, K: Strike Price, T: Time to expiration in years, r: risk free interest rate, vega: volatility, option: call or put
class BsOption:
    def __init__(self, S, K, T, r, sigma, q=0):
        self.S = S
        self.K = K
        self.T = T
        self.r = r 
        self.sigma = sigma
        self.q = q
        
    
    @staticmethod
    def N(x):
        return norm.cdf(x)
    
    def d1(self):
        return (np.log(self.S/self.K) + (self.r -self.q + self.sigma**2/2)*self.T) \
                                / (self.sigma*np.sqrt(self.T))
    
    def d2(self):
        return self.d1() - self.sigma*np.sqrt(self.T)
    
    def _call_value(self):
        return self.S*np.exp(-self.q*self.T)*self.N(self.d1()) - \
                    self.K*np.exp(-self.r*self.T) * self.N(self.d2())
                    
    def _put_value(self):
        return self.K*np.exp(-self.r*self.T) * self.N(-self.d2()) -\
                self.S*np.exp(-self.q*self.T)*self.N(-self.d1())
    def ind_call_value(self):
        return (self.S*np.exp(-self.q*self.T)*self.N(self.d1()) - \
                    self.K*np.exp(-self.r*self.T) * self.N(self.d2()))-1.92
                    
    def ind_put_value(self):
        return (self.K*np.exp(-self.r*self.T) * self.N(-self.d2()) -\
                self.S*np.exp(-self.q*self.T)*self.N(-self.d1()))-1.92
    
    def price(self, type_ = 'C'):
        if type_ == 'C':
            return self._call_value()
        if type_ == 'P':
            return self._put_value() 
        if type_ == 'B':
            return  {'call': self._call_value(), 'put': self._put_value()}
        else:
            raise ValueError('Unrecognized type')
    
    def indianprice(self, type_ = 'C'):
        if type_ == 'C':
            return self.ind_call_value()
        if type_ == 'P':
            return self.ind_put_value() 
        if type_ == 'B':
            return  {'call': self.ind_call_value(), 'put': self.ind_put_value()}
        else:
            raise ValueError('Unrecognized type')


In [37]:
import numpy as np1
#from nsetools import Nse
nse = Nse()
date = str(input("Enter the Expiration date in the format DD-Month-YYYY"))
K = np1.arange(20500,23500,50)
r = 0.0787
T = 3/365
sigma = 15/100
S = 22511


In [38]:
import pandas as pd
aDict = {}
aDict['Underlying Price'] = []
aDict['Strike Price'] = []
# aDict['Interest Rate'] = []
# aDict['Time to Expiration'] = []
# aDict['Volatility Predicition'] = []
aDict['Call Theo'] = []
aDict['Call Value'] = []
aDict['Put Theo'] = []
aDict ['Put Value'] = []
for k in K:
    aDict['Underlying Price'].append(S)
    aDict['Strike Price'].append(k)
    # aDict['Interest Rate'].append(round(r*100,2))
    # aDict['Time to Expiration'].append(T*365)
    aDict['Call Theo'].append(round(BsOption(S, k, T, r, sigma).price('C'),2))
    aDict['Put Theo'].append(round(BsOption(S, k, T, r, sigma).price('P'),2))

In [40]:
import requests
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/61.0.3163.100 Safari/537.36','Accept-Encoding': 'gzip, deflate, br','Accept-Language': 'en-US,en;q=0.9,hi;q=0.8'}
url = "https://www.nseindia.com/api/option-chain-indices?symbol=NIFTY"
json_obj = requests.get(url, headers = headers).json()
aDict['Call Value'] = []
aDict ['Put Value'] = []
for item in json_obj['records']['data']:
    if item['expiryDate'] == date:
        for k in range(len(aDict['Strike Price'])):
            if aDict['Strike Price'][k] == item['CE']['strikePrice']:
                aDict['Call Value'].append(item['CE']['lastPrice'])
                aDict['Put Value'].append(item['PE']['lastPrice'])

In [41]:
df = pd.DataFrame(aDict)
df["Call Edge"] = df['Call Theo'] - df['Call Value']
df["Put Edge"] = df['Put Theo'] - df['Put Value']
df.loc[:,['Underlying Price','Strike Price','Call Theo','Call Value','Call Edge','Put Theo','Put Value','Put Edge']]

,Underlying Price,Strike Price,Call Theo,Call Value,Call Edge,Put Theo,Put Value,Put Edge
0,22511,20500,2024.26,1984.60,39.66,0.00,1.45,-1.45
1,22511,20550,1974.29,1852.00,122.29,0.00,1.80,-1.80
2,22511,20600,1924.32,1864.15,60.17,0.00,1.65,-1.65
3,22511,20650,1874.35,1755.95,118.40,0.00,1.85,-1.85
4,22511,20700,1824.39,1749.35,75.04,0.00,1.80,-1.80
5,22511,20750,1774.42,1701.85,72.57,0.00,1.95,-1.95
6,22511,20800,1724.45,1710.00,14.45,0.00,1.90,-1.90
7,22511,20850,1674.48,1495.55,178.93,0.00,2.20,-2.20
8,22511,20900,1624.51,1259.00,365.51,0.00,2.15,-2.15
9,22511,20950,1574.55,1458.95,115.60,0.00,2.20,-2.20
